In [1]:
from collections import defaultdict
from datetime import datetime, timedelta
import os
from copy import deepcopy
import yaml

from clickhouse_driver import Client

#from src.cg import CoinGeckoAPI
#from src.handle_portfolio import get_coin_date_out, get_coin_date_enter, get_date_lst_coins
#from src.get_data import get_current_price_vol_cg, GetRawData, query_get_coins_volume_per_day, get_ndau_cur_day
#from src.calc_metrics import (
#    percentage_wrt_portfolio, postprocess_metric, iterator_window_df,
#    roi_enter, get_corr, get_beta, history_diff_trend
#)
from google_sheet import write_df, write_update_time, write_data_to_gs_no_resize
from tqdm.notebook import tqdm

from sklearn.linear_model import LinearRegression
import pandas as pd
import numpy as np
import pygsheets
import scipy
from scipy import stats



from datetime import datetime, timedelta
import time

In [2]:
def get_config(
        path_dir_main_config: str,
        path_sensitive_config: str
):
    path_main_config = os.path.join(path_dir_main_config, 'config.yaml')
    path_input_csv = os.path.join(path_dir_main_config, 'input.csv')
    
    with open(path_main_config,'r') as stream:
        main_config = yaml.safe_load(stream)

    with open(path_sensitive_config,'r') as stream:
        conf = yaml.safe_load(stream)
        cli_creds = conf['clickhouse']
        cg_api_key = conf['cg-api-key']
    
    input_p = pd.read_csv(path_input_csv)
    input_p = input_p.set_index('coingecko_id')
    return input_p, main_config, cli_creds, cg_api_key


def setup_logging():
    file_path = os.path.dirname(__file__)
    logging.basicConfig(
        filename=os.path.join(file_path, "main.log"),
        filemode='a',
        level=logging.INFO,
        format='%(asctime)s %(levelname)-8s %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S'
    )
    
def get_enter_prices(input_c):
    buy_prices_usdt = input_p['price_USDT'].to_dict()
    buy_prices_btc = input_p['price_BTC'].to_dict()
    buy_prices_eth = input_p['price_ETH'].to_dict()
    return buy_prices_usdt, buy_prices_btc, buy_prices_eth

def get_info(input_c):
    info = input_p[['ticker', 'price_USDT', 'price_BTC', 'price_ETH', 'amount_usdt', 'dt']] #.set_index('ticker')
    return info

In [3]:
input_p, main_config, cli_creds, cg_api_key = get_config('configs/config_5_25_03_2023/', 'configs/sensitive.yaml')

cli = Client(
    host=main_config['clickhouse']['client']['host'],
    user=cli_creds['user'],
    password=cli_creds['password']
        #settings=main_config['clickhouse']['client']['settings']
    )

cli.execute('select now();')


def init_():
    portfolio_members = pd.DataFrame(
        columns = [
            'dt',
            'coins'
        ]

    )

    unrealized = pd.DataFrame(
        columns = [
            'coingecko_id', 'dt_in', 
            'price_in_usd', 'price_in_btc', 'price_in_eth',
            'usd_in', 'btc_in', 'eth_in'

        ]
    )

    realized = pd.DataFrame(
        columns = [
            'coingecko_id', 'dt_in', 'dt_out',
            'price_in_usd', 'price_in_btc', 'price_in_eth',
            'usd_in', 'btc_in', 'eth_in',

            'price_out_usd', 'price_out_btc', 'price_out_eth',
            'usd_out', 'btc_out', 'eth_out',

            'fixed_usd', 'fixed_btc', 'fixed_eth',

        ]
    )
    
    return portfolio_members, unrealized, realized

In [4]:
actual_coins = cli.query_dataframe('select * from DEV_ODS_CROSS_BC_RESOURCES.SELECTED_FOR_DASH_TOKENS_V')
coins_today = actual_coins['coingecko_id'].values.tolist()
coins_today.remove('arbitrum')
coins_today.remove('ethereum')
coins_today.remove('bitcoin')

In [5]:
btc_sum = cli.query_dataframe(
    '''
        SELECT *
        FROM DEV_DM_ANALYTCS.EMA200_BTC_SUMMARY
        WHERE 
            coingecko_id in {}
        ORDER BY tech_load_ts asc
    '''.format(
        coins_today
    )
)

btc_sum['dt'] = pd.to_datetime(btc_sum['tech_load_ts']).dt.date - timedelta(days=1) #
btc_sum = btc_sum.groupby(['coingecko_id', 'dt']).first().reset_index()
btc_sum['dt'] = btc_sum['dt'].astype(str)

btc_sum = btc_sum[btc_sum['dt'] >= '2023-03-24']


In [6]:

def generate_dt_range(first_dt, last_dt):
    idx_dt = pd.date_range(first_dt, last_dt)
    return idx_dt

dt_range = generate_dt_range('2023-03-24', '2023-04-11')

lst_df = []
for c, df in btc_sum.groupby('coingecko_id'):
    df = df.set_index(pd.to_datetime(df['dt']))
    df = df.reindex(dt_range)
    df = df.fillna(method='ffill')
    lst_df.append(df.reset_index())
btc_sum = pd.concat(lst_df, axis=0)
btc_sum['dt'] = btc_sum['index'].astype(str)
btc_sum = btc_sum.drop(columns=['index'])
btc_sum.loc[btc_sum['coingecko_id'] == 'kwenta', 'days_above_200EMA'] = 2

btc_sum_input = btc_sum[btc_sum['dt'] == '2023-03-25']
btc_sum = btc_sum[btc_sum['dt'] > '2023-03-25']

#get_input
btc_sum_input[btc_sum_input['days_above_200EMA'] > 0]['coingecko_id'].values

array(['3xcalibur', 'camelot-token', 'floki', 'gains-network', 'gmx',
       'hamachi-finance', 'hop-protocol', 'immutable-x', 'joe', 'kaspa',
       'kwenta', 'magic', 'multichain', 'nakamoto-games', 'oath', 'okb',
       'phoenix-global', 'plutusdao', 'radiant-capital', 'ssv-network',
       'velodrome-finance'], dtype=object)

In [7]:
btc_sum = btc_sum.reset_index(drop=True)

In [8]:
btc_sum.iloc[-19:]

,coingecko_id,dt,period,trend_200EMA_180d,trend_200EMA_90d,price_above_200EMA_percentage,days_above_200EMA,tech_load_ts,days_above_200EMA_dt_candle_close,dt_candle_close
746,vesta-finance,2023-04-10,24h,-0.855520,-0.574804,-33.325642,-11.0,2023-04-11 09:12:36,-11.0,2023-04-10
747,vesta-finance,2023-04-11,24h,-0.855520,-0.574804,-34.910233,-12.0,2023-04-12 10:34:26,-12.0,2023-04-11
748,winr-protocol,2023-03-26,1h,0.500039,0.500039,3.618201,1.0,2023-03-27 03:06:53,0.0,1970-01-01
749,winr-protocol,2023-03-27,1h,0.724942,0.724942,-2.081192,-1.0,2023-03-28 03:06:06,0.0,1970-01-01
750,winr-protocol,2023-03-28,1h,0.582821,0.582821,6.378032,1.0,2023-03-29 03:05:29,0.0,1970-01-01
751,winr-protocol,2023-03-29,1h,0.793000,0.935263,9.203882,2.0,2023-03-30 03:06:09,0.0,1970-01-01
752,winr-protocol,2023-03-30,1h,0.933951,0.959157,4.644594,1.0,2023-03-31 03:06:32,0.0,1970-01-01
753,winr-protocol,2023-03-31,1h,0.893373,0.776053,2.671848,1.0,2023-04-01 02:31:52,0.0,1970-01-01
754,winr-protocol,2023-04-01,1h,0.921678,0.846071,9.144352,1.0,2023-04-02 03:06:34,0.0,1970-01-01
755,winr-protocol,2023-04-02,1h,0.858518,0.663633,0.901880,1.0,2023-04-03 03:06:31,0.0,1970-01-01


In [9]:
btc_sum.loc[766] = ['winr-protocol', '2023-03-27', '1h', 0.724942, 0.724942, 1, 1.0, '2023-03-28 03:06:06', 0, '1970-01-01']

In [11]:
btc_sum['dt_candle_close'] = pd.to_datetime(btc_sum['dt_candle_close']).dt.date.astype(str)

btc_sum.loc[
    btc_sum['dt_candle_close'] != '1970-01-01',
    'days_above_200EMA'
] = btc_sum.loc[
    btc_sum['dt_candle_close'] != '1970-01-01',
    'days_above_200EMA_dt_candle_close'
]

In [12]:
btc_sum

,coingecko_id,dt,period,trend_200EMA_180d,trend_200EMA_90d,price_above_200EMA_percentage,days_above_200EMA,tech_load_ts,days_above_200EMA_dt_candle_close,dt_candle_close
0,3xcalibur,2023-03-26,8h,0.635184,0.635184,7.913414,1.0,2023-03-27 03:06:53,0.0,1970-01-01
1,3xcalibur,2023-03-27,12h,-0.916293,-0.916293,-35.008919,-2.0,2023-03-28 03:06:06,0.0,1970-01-01
2,3xcalibur,2023-03-28,12h,-0.920708,-0.920708,-19.807335,-3.0,2023-03-29 03:05:29,0.0,1970-01-01
3,3xcalibur,2023-03-29,12h,-0.924783,-0.924783,-19.849369,-4.0,2023-03-30 03:06:09,0.0,1970-01-01
4,3xcalibur,2023-03-30,12h,-0.928551,-0.928551,-27.730942,-5.0,2023-03-31 03:06:32,0.0,1970-01-01
...,...,...,...,...,...,...,...,...,...,...
761,winr-protocol,2023-04-08,1h,-0.970163,-1.000000,-18.578703,-4.0,2023-04-09 03:06:03,0.0,1970-01-01
762,winr-protocol,2023-04-09,1h,-0.995489,-1.000000,-14.976006,-5.0,2023-04-10 03:06:06,0.0,1970-01-01
763,winr-protocol,2023-04-10,1h,-0.997352,-1.000000,-14.969000,-9.0,2023-04-11 09:12:36,-9.0,2023-04-10
764,winr-protocol,2023-04-11,1h,-0.999834,-1.000000,-14.501753,-10.0,2023-04-12 10:34:26,-10.0,2023-04-11


In [13]:

all_action_coins = coins_today #list(set(coins_this_day).union(prev_coins))
prices = cli.query_dataframe(
    '''
    SELECT 
    coingecko_id, price, date_of_candle_close as dt_candle_close
    FROM DEV_ODS_CG_MARKET.DEV_ALL_TOKEN_METRICS 
    where coingecko_id in {lst_coins}    
    '''.format(lst_coins = all_action_coins + ['bitcoin', 'ethereum'])

)
prices['dt_candle_close'] = prices['dt_candle_close'].astype(str)
prices = prices.drop_duplicates(subset=['coingecko_id', 'dt_candle_close'])

prices = prices[prices['dt_candle_close'] >= '2023-03-25']

prices[prices['coingecko_id'] == 'optimism']

,coingecko_id,price,dt_candle_close
3131,optimism,2.263170,2023-04-06
3132,optimism,2.266643,2023-04-07
3133,optimism,2.238477,2023-04-08
3134,optimism,2.288721,2023-04-09
3135,optimism,2.352827,2023-04-10
3243,optimism,2.298188,2023-04-11
13067,optimism,2.191602,2023-03-25
13068,optimism,2.269058,2023-03-26
13069,optimism,2.071993,2023-03-27
13070,optimism,2.197812,2023-03-28


In [14]:
#ReDo input
def input_(dt_input, first_day_coins, input_p, portfolio_members, unrealized, amount=10_000):
    all_coins = first_day_coins
    
    portfolio_members.loc[dt_input] = [dt_input, all_coins]
    
    print(all_coins)
    
    for cg_id in all_coins:
        unrealized.loc['{} {}'.format(cg_id, dt_input)] = {
            'coingecko_id': cg_id,
            'dt_in': dt_input,
            'price_in_usd': input_p.loc[cg_id, 'price_USDT'],
            'price_in_btc': input_p.loc[cg_id, 'price_BTC'],
            'price_in_eth': input_p.loc[cg_id, 'price_ETH'],
            
            'usd_in': amount,
            'btc_in': amount,
            'eth_in': amount
            
            
        }
        
    return unrealized, portfolio_members

fi_coins = btc_sum_input[btc_sum_input['days_above_200EMA'] > 0]['coingecko_id'].values.tolist()


portfolio_members, unrealized, realized = init_()
input_('2023-03-25', fi_coins, input_p, portfolio_members, unrealized);





['3xcalibur', 'camelot-token', 'floki', 'gains-network', 'gmx', 'hamachi-finance', 'hop-protocol', 'immutable-x', 'joe', 'kaspa', 'kwenta', 'magic', 'multichain', 'nakamoto-games', 'oath', 'okb', 'phoenix-global', 'plutusdao', 'radiant-capital', 'ssv-network', 'velodrome-finance']


In [15]:
portfolio_members

,dt,coins
2023-03-25,2023-03-25,"[3xcalibur, camelot-token, floki, gains-networ..."


In [16]:
def price_wrt_base_coin(price_ts, base_coin):
    price_ts_wrt = price_ts.merge(
        (price_ts.loc[price_ts['coingecko_id'] == base_coin, ['price', 'dt_candle_close']]
         .rename(columns={'price': f'price_{base_coin}'})
         ),
        on='dt_candle_close'
    )
    price_ts_wrt[f'price_wrt_{base_coin}'] = price_ts_wrt['price'] / price_ts_wrt[f'price_{base_coin}']

    return price_ts_wrt

In [17]:
prices_wrt_btc =  price_wrt_base_coin(prices, 'bitcoin')[['coingecko_id', 'dt_candle_close', 'price_wrt_bitcoin']]
price_wrt_eth =  price_wrt_base_coin(prices, 'ethereum')[['coingecko_id', 'dt_candle_close', 'price_wrt_ethereum']]
prices_wrt_usd = prices[['coingecko_id', 'dt_candle_close', 'price']].rename(columns={"price": 'price_wrt_usd'})


prices = pd.merge(
    pd.merge(
        prices_wrt_btc, price_wrt_eth, on=['coingecko_id','dt_candle_close']
    ), 
    prices_wrt_usd, on=['coingecko_id','dt_candle_close' ]
)
    

In [18]:
dict_above_ema = {}
for dt, df_dt in btc_sum.groupby('dt'):
    dict_above_ema[dt] = df_dt.loc[df_dt['days_above_200EMA'] > 0, 'coingecko_id'].values.tolist()
    

In [20]:

portfolio_coins = pd.DataFrame(columns=['dt_candle_close', 'coins_in_portolio'])
for dt, coins in dict_above_ema.items():
    portfolio_coins.loc[dt] = [dt, coins]




In [ ]:
#portfolio_coins['tech_load_ts'] = datetime.today()
#portfolio_coins['dt_candle_close'] = pd.to_datetime(portfolio_coins['dt_candle_close'])
    
#cli.execute('INSERT INTO DEV_PORTFOLIO_DYNAMIC_5_2023_03_26.PORTFOLIO_COINS VALUES;', portfolio_coins.to_dict('records'))

In [22]:
def handle_out(dt_out, coins_out, unrealized, realized, prices):
    for coin_out in coins_out:  
        if coin_out == 'optimism':
            print('out', dt_out)
        prices_in = unrealized.loc[(unrealized['coingecko_id'] == coin_out)].squeeze()
    
        dt_in = prices_in.loc['dt_in']
        price_in_usd = prices_in.loc['price_in_usd']
        price_in_btc = prices_in.loc['price_in_btc']
        price_in_eth = prices_in.loc['price_in_eth']
        usd_in = prices_in.loc['usd_in']
        btc_in = prices_in.loc['btc_in']
        eth_in = prices_in.loc['eth_in']

        prices_out = prices[
            (prices['coingecko_id'] == coin_out) & (prices['dt_candle_close'] == dt_out)
        ].squeeze()
        
        

        price_out_usd = prices_out.loc['price_wrt_usd']
        price_out_btc = prices_out.loc['price_wrt_bitcoin']
        price_out_eth = prices_out.loc['price_wrt_ethereum']
        usd_out = usd_in / price_in_usd * price_out_usd
        btc_out = btc_in / price_in_btc * price_out_btc
        eth_out = eth_in / price_in_eth * price_out_eth
        
        fixed_usd = usd_out - usd_in
        fixed_btc = btc_out - btc_in
        fixed_eth = eth_out - eth_in
        
        

        realized.loc['{} {} {}'.format(coin_out, dt_in, dt_out)] = {
            'coingecko_id': coin_out,
            'dt_in': dt_in,
            'dt_out': dt_out,
            'price_in_usd': price_in_usd,
            'price_in_btc': price_in_btc,
            'price_in_eth': price_in_eth,
            'usd_in': usd_in,
            'btc_in': btc_in,
            'eth_in': eth_in,
    
            'price_out_usd': price_out_usd,
            'price_out_btc': price_out_btc,
            'price_out_eth': price_out_eth,
            'usd_out': usd_out,
            'btc_out': btc_out,
            'eth_out': eth_out,
            
            'fixed_usd': fixed_usd,
            'fixed_btc': fixed_btc,
            'fixed_eth': fixed_eth,
        }
        
        unrealized = unrealized.drop('{} {}'.format(coin_out, dt_in))
    
    return unrealized, realized

def handle_in(dt_calc, coins_in, unrealized, prices, amount=10_000):
    dt_in = dt_calc
    for coin_in in coins_in:
        if coin_in == 'optimism':
            print('in', dt_in)
        
        
        prices_in = prices[
                (prices['coingecko_id'] == coin_in) & (prices['dt_candle_close'] == dt_in)
            ].squeeze()

        unrealized.loc['{} {}'.format(coin_in, dt_in)] = {
            'coingecko_id': coin_in,
            'dt_in': dt_in, 
            'price_in_usd': prices_in.loc['price_wrt_usd'], 
            'price_in_btc': prices_in.loc['price_wrt_bitcoin'], 
            'price_in_eth': prices_in.loc['price_wrt_ethereum'], 
            
            'usd_in': amount,
            'btc_in': amount,
            'eth_in': amount
        }
        
    return unrealized

def make_snapshot(dt_calc, unrealized, prices):
    pass

def step_day(dt_calc, coins_this_day, portfolio_members, unrealized, realized, prices):
    
    coins_this_day = dict_above_ema[dt_calc]
    prev_coins = portfolio_members.iloc[-1]['coins']

    coins_out = [coin for coin in prev_coins if coin not in coins_this_day]
    coins_in =  [coin for coin in coins_this_day if coin not in prev_coins]
    
    unrealized, realized = handle_out(dt_calc, coins_out, unrealized, realized, prices)
    
    snapshot = pd.DataFrame(columns=realized.columns)
    unrealized_before = unrealized[unrealized['dt_in'] < dt_calc]
    coins = unrealized_before['coingecko_id'].values.tolist()
    _, snapshot = handle_out(dt_calc, coins, unrealized_before, snapshot, prices)

    unrealized = handle_in(dt_calc, coins_in, unrealized, prices)
    portfolio_members.loc[dt_calc] = [dt_calc, coins_this_day]
    
    
    
    return portfolio_members, unrealized, realized, snapshot

In [23]:
portfolio_members, unrealized, realized = init_()
input_('2023-03-25', fi_coins, input_p, portfolio_members, unrealized);

['3xcalibur', 'camelot-token', 'floki', 'gains-network', 'gmx', 'hamachi-finance', 'hop-protocol', 'immutable-x', 'joe', 'kaspa', 'kwenta', 'magic', 'multichain', 'nakamoto-games', 'oath', 'okb', 'phoenix-global', 'plutusdao', 'radiant-capital', 'ssv-network', 'velodrome-finance']


In [25]:
lst_snapshots = []
lst_realized = []
lst_unrealized = []
dts =  [
    '2023-03-26', '2023-03-27', '2023-03-28', '2023-03-29', '2023-03-30',
    '2023-03-31', '2023-04-01', '2023-04-02', '2023-04-03', '2023-04-04',
    '2023-04-05', '2023-04-06', '2023-04-07', '2023-04-08', '2023-04-09',
    '2023-04-10', '2023-04-11'

]
for dt_calc in dts:
    print(dt_calc)
    portfolio_members, unrealized, realized, snapshot = (
        step_day(dt_calc, dict_above_ema[dt_calc], portfolio_members.copy(), unrealized.copy(), realized.copy(), prices)
    )
    
    lst_snapshots.append(snapshot)
    lst_realized.append(realized)
    lst_unrealized.append(unrealized)
    

snapshots = pd.concat(lst_snapshots)
#realized['roi_btc'] = (realized['btc_out'] - realized['btc_in']) / realized['btc_in'] * 100

2023-03-26
in 2023-03-26
2023-03-27
out 2023-03-27
2023-03-28
in 2023-03-28
2023-03-29
out 2023-03-29
2023-03-30
in 2023-03-30
2023-03-31
out 2023-03-31
2023-04-01
out 2023-04-01
2023-04-02
out 2023-04-02
2023-04-03
2023-04-04
2023-04-05
in 2023-04-05
2023-04-06
out 2023-04-06
2023-04-07
out 2023-04-07
2023-04-08
out 2023-04-08
2023-04-09
out 2023-04-09
2023-04-10
out 2023-04-10
2023-04-11
out 2023-04-11


In [26]:
lst_realized[1]

,coingecko_id,dt_in,dt_out,price_in_usd,price_in_btc,price_in_eth,usd_in,btc_in,eth_in,price_out_usd,price_out_btc,price_out_eth,usd_out,btc_out,eth_out,fixed_usd,fixed_btc,fixed_eth
gmx 2023-03-25 2023-03-26,gmx,2023-03-25,2023-03-26,69.717961,0.002520,0.039748,10000,10000,10000,69.355940,0.002473,0.039033,9948.073403,9813.910559,9819.947827,-51.926597,-186.089441,-180.052173
3xcalibur 2023-03-25 2023-03-27,3xcalibur,2023-03-25,2023-03-27,0.247817,0.000009,0.000141,10000,10000,10000,0.217012,0.000008,0.000126,8756.939552,8914.119421,8934.259691,-1243.060448,-1085.880579,-1065.740309
immutable-x 2023-03-25 2023-03-27,immutable-x,2023-03-25,2023-03-27,1.054206,0.000038,0.000601,10000,10000,10000,1.023711,0.000038,0.000595,9710.728640,9885.028239,9907.362149,-289.271360,-114.971761,-92.637851
optimism 2023-03-26 2023-03-27,optimism,2023-03-26,2023-03-27,2.269058,0.000081,0.001277,10000,10000,10000,2.071993,0.000076,0.001205,9131.512437,9422.490245,9437.973112,-868.487563,-577.509755,-562.026888
phoenix-global 2023-03-25 2023-03-27,phoenix-global,2023-03-25,2023-03-27,1.062616,0.000038,0.000606,10000,10000,10000,1.000657,0.000037,0.000582,9416.913052,9585.938903,9607.597060,-583.086948,-414.061097,-392.402940
vesta-finance 2023-03-26 2023-03-27,vesta-finance,2023-03-26,2023-03-27,0.638791,0.000023,0.000360,10000,10000,10000,0.534815,0.000020,0.000311,8372.299141,8639.084438,8653.280026,-1627.700859,-1360.915562,-1346.719974


In [88]:
#for realized, dt_c in zip(lst_realized, dts):

realized['tech_load_ts'] = datetime.today()
realized['dt_in'] = pd.to_datetime(realized['dt_in'])
realized['dt_out'] = pd.to_datetime(realized['dt_out'])
realized['dt_candle_close'] = realized['dt_out']
num = cli.execute('INSERT INTO DEV_PORTFOLIO_DYNAMIC_5_2023_03_26.REALIZED_HISTORY VALUES;', realized.to_dict('records'))
print(num)

22


In [95]:
print('\n'.join(unrealized.columns.tolist()))
for unrealized, dt_c in zip(lst_unrealized, dts):
    unrealized['tech_load_ts'] = datetime.today()
    unrealized['dt_candle_close'] = datetime.strptime(dt_c, '%Y-%m-%d')
    unrealized['dt_in'] = pd.to_datetime(unrealized['dt_in'])
    #unrealized['dt_out'] = pd.to_datetime(unrealized['dt_out'])
    num = cli.execute('INSERT INTO DEV_PORTFOLIO_DYNAMIC_5_2023_03_26.UNREALIZED_HISTORY VALUES;', unrealized.to_dict('records'))
    print(num)

23
19
23
21
23
23
23
20
20
22
25
22
20
20


In [98]:
#print('\n'.join(snapshots.columns.tolist()))
for snapshots, dt_c in zip(lst_snapshots, dts):
    snapshots['tech_load_ts'] = datetime.today()
    snapshots['dt_candle_close'] = datetime.strptime(dt_c, '%Y-%m-%d')
    snapshots['dt_in'] = pd.to_datetime(snapshots['dt_in'])
    snapshots['dt_out'] = pd.to_datetime(snapshots['dt_out'])
    num = cli.execute('INSERT INTO DEV_PORTFOLIO_DYNAMIC_5_2023_03_26.SNAPSHOTS VALUES;', snapshots.to_dict('records'))
    print(num)

coingecko_id
dt_in
dt_out
price_in_usd
price_in_btc
price_in_eth
usd_in
btc_in
eth_in
price_out_usd
price_out_btc
price_out_eth
usd_out
btc_out
eth_out
fixed_usd
fixed_btc
fixed_eth
20
18
19
21
21
22
22
20
18
20
22
22
19
19


In [27]:
realized_dt = realized.groupby('dt_out').agg({
    'usd_in': 'sum',
    'btc_in': 'sum',
    'eth_in': 'sum',
    'usd_out': 'sum',
    'btc_out': 'sum',
    'eth_out': 'sum'    
})

snapshots_dt = snapshots.groupby('dt_out').agg({
    'usd_in': 'sum',
    'btc_in': 'sum',
    'eth_in': 'sum',
    'usd_out': 'sum',
    'btc_out': 'sum',
    'eth_out': 'sum'    
})

In [28]:
realized_dt

,usd_in,btc_in,eth_in,usd_out,btc_out,eth_out
dt_out,,,,,,
2023-03-26,10000,10000,10000,9948.073403,9813.910559,9819.947827
2023-03-27,50000,50000,50000,45388.392822,46446.661245,46540.472038
2023-03-29,20000,20000,20000,20033.020242,19254.383172,19815.531071
2023-03-31,10000,10000,10000,9144.387934,8991.859005,8988.416531
2023-04-01,10000,10000,10000,9562.238901,9568.224781,9567.587175
2023-04-02,30000,30000,30000,29248.741016,28655.967210,28866.901340
2023-04-03,20000,20000,20000,18721.876064,18747.907848,18252.836573
2023-04-06,30000,30000,30000,28264.536986,28244.880658,27718.184976
2023-04-07,30000,30000,30000,25680.373603,25473.496790,25029.235872


In [29]:

realized_dt.loc['2023-03-28'] = [np.nan for i in range(len(realized_dt.columns))]
realized_dt.loc['2023-03-30'] = [np.nan for i in range(len(realized_dt.columns))]
realized_dt.loc['2023-04-04'] = [np.nan for i in range(len(realized_dt.columns))]
realized_dt.loc['2023-04-05'] = [np.nan for i in range(len(realized_dt.columns))]
realized_dt = realized_dt.sort_index()
realized_dt = realized_dt.fillna(value=0) #method='ffill')

In [30]:
realized_dt = realized_dt.expanding().sum()

In [31]:
realized_dt

,usd_in,btc_in,eth_in,usd_out,btc_out,eth_out
dt_out,,,,,,
2023-03-26,10000.0,10000.0,10000.0,9948.073403,9813.910559,9819.947827
2023-03-27,60000.0,60000.0,60000.0,55336.466226,56260.571804,56360.419865
2023-03-28,60000.0,60000.0,60000.0,55336.466226,56260.571804,56360.419865
2023-03-29,80000.0,80000.0,80000.0,75369.486468,75514.954976,76175.950936
2023-03-30,80000.0,80000.0,80000.0,75369.486468,75514.954976,76175.950936
2023-03-31,90000.0,90000.0,90000.0,84513.874403,84506.813981,85164.367467
2023-04-01,100000.0,100000.0,100000.0,94076.113304,94075.038762,94731.954641
2023-04-02,130000.0,130000.0,130000.0,123324.854320,122731.005972,123598.855981
2023-04-03,150000.0,150000.0,150000.0,142046.730384,141478.913820,141851.692554


In [32]:
def get_roi(df):
    df = deepcopy(df)
    df['roi_usd'] = (df['usd_out'] - df['usd_in']) / df['usd_in'] * 100
    df['roi_btc'] = (df['btc_out'] - df['btc_in']) / df['btc_in'] * 100
    df['roi_eth'] = (df['eth_out'] - df['eth_in']) / df['eth_in'] * 100
    return df

In [33]:
snapshots_dt

,usd_in,btc_in,eth_in,usd_out,btc_out,eth_out
dt_out,,,,,,
2023-03-26,200000,200000,200000,211736.598495,208881.052192,209009.550496
2023-03-27,180000,180000,180000,180747.538013,184117.869797,184528.099971
2023-03-28,190000,190000,190000,199264.183516,201977.194655,196707.462250
2023-03-29,210000,210000,210000,230516.919160,224314.044435,225379.447985
2023-03-30,210000,210000,210000,227262.864236,223922.918764,222513.865526
2023-03-31,220000,220000,220000,242836.883323,235421.603569,233918.668926
2023-04-01,220000,220000,220000,242247.975331,235014.145694,233487.981900
2023-04-02,200000,200000,200000,219374.953498,215072.398717,214470.951254
2023-04-03,180000,180000,180000,195292.010250,194257.426995,189536.425452


In [34]:
tog = pd.concat([snapshots_dt, realized_dt])
tog = tog.groupby(tog.index).sum()

get_roi(tog)

,usd_in,btc_in,eth_in,usd_out,btc_out,eth_out,roi_usd,roi_btc,roi_eth
dt_out,,,,,,,,,
2023-03-26,210000.0,210000.0,210000.0,221684.671899,218694.962752,218829.498324,5.564129,4.140458,4.204523
2023-03-27,240000.0,240000.0,240000.0,236084.004239,240378.441601,240888.519836,-1.631665,0.157684,0.370217
2023-03-28,250000.0,250000.0,250000.0,254600.649742,258237.766460,253067.882115,1.840260,3.295107,1.227153
2023-03-29,290000.0,290000.0,290000.0,305886.405628,299828.999411,301555.398921,5.478071,3.389310,3.984620
2023-03-30,290000.0,290000.0,290000.0,302632.350704,299437.873739,298689.816461,4.355983,3.254439,2.996488
2023-03-31,310000.0,310000.0,310000.0,327350.757725,319928.417550,319083.036393,5.597019,3.202715,2.930012
2023-04-01,320000.0,320000.0,320000.0,336324.088635,329089.184456,328219.936541,5.101278,2.840370,2.568730
2023-04-02,330000.0,330000.0,330000.0,342699.807818,337803.404689,338069.807235,3.848427,2.364668,2.445396
2023-04-03,330000.0,330000.0,330000.0,337338.740634,335736.340815,331388.118006,2.223861,1.738285,0.420642


In [35]:
snapshots['roi_btc'] = (snapshots['btc_out'] - snapshots['btc_in']) / snapshots['btc_in'] * 100
snapshots['roi_usd'] = (snapshots['usd_out'] - snapshots['usd_in']) / snapshots['usd_in'] * 100
snapshots['roi_eth'] = (snapshots['eth_out'] - snapshots['eth_in']) / snapshots['eth_in'] * 100

roi_btc = pd.pivot(snapshots, index='dt_out', columns='coingecko_id', values='roi_btc')
roi_eth = pd.pivot(snapshots, index='dt_out', columns='coingecko_id', values='roi_eth')
roi_usd = pd.pivot(snapshots, index='dt_out', columns='coingecko_id', values='roi_usd')

In [36]:
#add empty rows

In [37]:
realized['roi_btc'] = (realized['btc_out'] - realized['btc_in']) / realized['btc_in'] * 100
realized['roi_usd'] = (realized['usd_out'] - realized['usd_in']) / realized['usd_in'] * 100
realized['roi_eth'] = (realized['eth_out'] - realized['eth_in']) / realized['eth_in'] * 100

realized_btc = pd.pivot(realized, index='dt_out', columns='coingecko_id', values='roi_btc')
realized_usd = pd.pivot(realized, index='dt_out', columns='coingecko_id', values='roi_usd')
realized_eth = pd.pivot(realized, index='dt_out', columns='coingecko_id', values='roi_eth')


In [38]:
#input_p, main_config, cli_creds, cg_api_key = get_config('configs/config_5_25_03_2023/', 'configs/sensitive.yaml')
info = get_info(input_p)
info

,ticker,price_USDT,price_BTC,price_ETH,amount_usdt,dt
coingecko_id,,,,,,
3xcalibur,XCAL,0.247817,8.956111e-06,1.412884e-04,1,2023-03-25
aleph-zero,AZERO,1.464429,5.292441e-05,8.349164e-04,1,2023-03-25
alienfi,ALIEN,0.524992,1.897318e-05,2.993141e-04,1,2023-03-25
allianceblock-nexera,NXRA,0.070561,2.550076e-06,4.022908e-05,1,2023-03-25
aptos,APT,11.665619,4.215951e-04,6.650932e-03,1,2023-03-25
astar,ASTR,0.061291,2.215056e-06,3.494393e-05,1,2023-03-25
aura-finance,AURA,2.883135,1.041964e-04,1.643765e-03,1,2023-03-25
benddao,BEND,0.012567,4.541571e-07,7.164619e-06,1,2023-03-25
bone-shibaswap,BONE,1.121027,4.051388e-05,6.391324e-04,1,2023-03-25


In [39]:
def cg2ticker(metric_df, info):
    return metric_df.rename(columns=info['ticker'].to_dict())

roi_btc = cg2ticker(roi_btc, info)
roi_eth = cg2ticker(roi_usd, info)
roi_usd = cg2ticker(roi_usd, info)

realized_btc = cg2ticker(realized_btc, info)
realized_eth = cg2ticker(realized_usd, info)
realized_usd = cg2ticker(realized_usd, info)


In [40]:
gc = pygsheets.authorize(
    service_file=main_config['google-sheet']['path-to-key-file']
)
sh = gc.open_by_key(main_config['google-sheet']['sheet-id'])



In [41]:
info.index.name='coin'
write_df(info, sh, 'info', 1, 'days', to_UTC=True)

In [42]:

write_df(roi_usd, sh, 'roi_usd', 1, 'days', to_UTC=True)
write_df(roi_btc, sh, 'roi_btc', 1, 'days', to_UTC=True)
write_df(roi_eth, sh, 'roi_eth', 1, 'days', to_UTC=True)


#write_df(ndau_sum, sh, 'ndau_sum', 1, 'days', to_UTC=True)
#write_df(volume_sum, sh, 'volume_sum', 1, 'days', to_UTC=True)

In [43]:
write_df(realized_usd, sh, 'fixed_usd', 1, 'days', to_UTC=True)
write_df(realized_btc, sh, 'fixed_btc', 1, 'days', to_UTC=True)
write_df(realized_eth, sh, 'fixed_eth', 1, 'days', to_UTC=True)

In [44]:
prices_volumes = cli.query_dataframe(
    '''
    SELECT 
    coingecko_id, price, total_volume, date_of_candle_close as dt_candle_close
    FROM DEV_ODS_CG_MARKET.DEV_ALL_TOKEN_METRICS 
    where coingecko_id in {lst_coins}    
    '''.format(lst_coins = all_action_coins + ['bitcoin', 'ethereum'])

)
prices_volumes['dt_candle_close'] = prices_volumes['dt_candle_close'].astype(str)
prices_volumes = prices_volumes.drop_duplicates(subset=['coingecko_id', 'dt_candle_close'])

In [45]:
def iterator_window_df(df, first_date, window_length, last_date):
    window_end = datetime.strptime(first_date, '%Y-%m-%d')
    while window_end.strftime('%Y-%m-%d') <= last_date:
        window_start = window_end - timedelta(days=window_length)
        df_window = df[
            (df.index >= window_start.strftime('%Y-%m-%d')) &
            (df.index <= window_end.strftime('%Y-%m-%d'))
            ]

        yield window_end, df_window
        window_end += timedelta(days=1)

In [47]:
lst_coins = coins_today
first_enter = '2023-03-26'
date_calc = '2023-04-11'
period=30

In [48]:
prices_pivoted = prices_volumes.pivot(index='dt_candle_close', columns='coingecko_id', values='price')

In [49]:
base_coin = 'bitcoin'
df_beta_eth = pd.DataFrame(columns = lst_coins+[base_coin])
for dt, df_window in iterator_window_df(prices_pivoted, first_enter, period, date_calc):
    daily_roi = (
        (df_window - df_window.shift(1)) / df_window.shift(1)
    ).iloc[1:]
    beta = daily_roi.cov()[base_coin] / (daily_roi.std()[base_coin] ** 2)
    df_beta_eth.loc[dt.strftime('%Y-%m-%d')] = beta

In [50]:
base_coin = 'ethereum'
df_beta_btc = pd.DataFrame(columns = lst_coins+[base_coin])
for dt, df_window in iterator_window_df(prices_pivoted, first_enter, period, date_calc):
    daily_roi = (
        (df_window - df_window.shift(1)) / df_window.shift(1)
    ).iloc[1:]
    beta = daily_roi.cov()[base_coin] / (daily_roi.std()[base_coin] ** 2)
    df_beta_btc.loc[dt.strftime('%Y-%m-%d')] = beta

In [51]:
base_coin = 'ethereum'
df_corr_eth = pd.DataFrame(columns = lst_coins+[base_coin])
for dt, df_window in iterator_window_df(prices_pivoted, first_enter, 30, date_calc):
    df_corr_eth.loc[dt.strftime('%Y-%m-%d')] = df_window.corr().loc[base_coin]

In [52]:
base_coin = 'bitcoin'
df_corr_btc = pd.DataFrame(columns = lst_coins+[base_coin])
for dt, df_window in iterator_window_df(prices_pivoted, first_enter, 30, date_calc):
    df_corr_btc.loc[dt.strftime('%Y-%m-%d')] = df_window.corr().loc[base_coin]

In [53]:
for dt, df in portfolio_members.iloc[1:].iterrows():
    coin_in = set(portfolio_members.loc[dt, 'coins'])
    coin_not_in = set(df_corr_btc.columns).difference(set(coin_in)).difference({'bitcoin', 'ethereum'})
    df_corr_btc.loc[dt, coin_not_in] = np.nan
    df_beta_btc.loc[dt, coin_not_in] = np.nan
    df_corr_eth.loc[dt, coin_not_in] = np.nan
    df_beta_eth.loc[dt, coin_not_in] = np.nan
    

In [54]:
print('\n'.join([str(a) for a in df_beta_btc.mean(axis=1).values.tolist()]))

1.4026573105346578
1.236269267278457
1.346410006195958
1.2721850572970366
1.34323032707494
1.2903102682013652
1.2741919739528205
1.2540120019866212
1.3114978571896765
1.3453860361063026
1.3316468343860655
1.2825431402894412
1.227453860586396
1.2429471038779485
1.217368473036587
1.3441166333433558
1.223999308651243


In [55]:
print('\n'.join([str(a) for a in df_beta_eth.mean(axis=1).values.tolist()]))

1.3873742109207667
1.2575958516707186
1.328444363321571
1.2660596404727746
1.310561956250697
1.257936606472587
1.2523176783319427
1.2516169370413281
1.2795146171902196
1.3032990805462925
1.288875363812605
1.2547656398381615
1.1696876324477252
1.235932964617578
1.1787854932259725
1.2378656296625383
1.1379002924093218


In [56]:
print('\n'.join([str(a) for a in df_corr_eth.mean(axis=1).values.tolist()]))

0.5553619978186476
0.553289225306066
0.5457276241215784
0.5402772586685113
0.5449048493173884
0.5254637438527349
0.4877388210777738
0.4781124218956506
0.5157632487380707
0.48503091838155327
0.4593451706526824
0.4947469496574107
0.479500057171909
0.5080600307259309
0.4886586959746572
0.35720770344857533
0.2846966371410395


In [57]:
print('\n'.join([str(a) for a in df_corr_btc.mean(axis=1).values.tolist()]))

0.5524881790207691
0.5502671672041239
0.5269886158745818
0.5262394233537327
0.5273981093992021
0.5007845581554646
0.46948096821854396
0.47061607046776477
0.4931104515273182
0.4607949872519187
0.4306303234933686
0.47222692665104776
0.47553141623650624
0.5226763219180303
0.4943747180749167
0.3725170727527159
0.3033708465179443


In [ ]:
def cg2ticker(metric_df, info):
    return metric_df.rename(columns=info['ticker'].to_dict())

roi_btc = cg2ticker(roi_btc, info)
roi_eth = cg2ticker(roi_usd, info)
roi_usd = cg2ticker(roi_usd, info)

realized_btc = cg2ticker(realized_btc, info)
realized_eth = cg2ticker(realized_usd, info)
realized_usd = cg2ticker(realized_usd, info)


In [60]:
for dt, df in portfolio_members.iterrows():
    vol_days = prices_volumes[prices_volumes['dt_candle_close'] == dt]
    vol_days = vol_days[vol_days['coingecko_id'].isin(df['coins'])]
    print(vol_days['total_volume'].mean())



12784362.098432938
20329179.138140503
23108354.838708855
30208218.454191048
23249367.365573674
39231466.89088985


In [165]:
vol_days

,coingecko_id,ndau,dt_candle_close


In [64]:
ndau['dt_candle_close'] = ndau['dt_candle_close'].astype(str)
for dt, df in portfolio_members.iterrows():
    vol_days = ndau[ndau['dt_candle_close'] == dt]
    vol_days = vol_days[vol_days['coingecko_id'].isin(df['coins'])]
    #print(dt)
    print(vol_days['ndau'].mean())



156.95
264.90909090909093
196.11111111111111
241.04545454545453
184.6
344.1363636363636


In [62]:
def query_get_coins_ndau_per_day(coins_1d, ndau_table):
    q = '''
        SELECT 
            coingecko_id, sum(ndau) as ndau, date_ndau as dt_candle_close
        FROM 
            {ndau_table}
        WHERE
            coingecko_id in {lst_coins}
        GROUP BY
            coingecko_id, date_ndau
        '''.format(
            lst_coins=tuple(coins_1d),
            ndau_table=ndau_table
        )
    return q

In [63]:
ndau = cli.query_dataframe(query_get_coins_ndau_per_day(coins_today, 'DEV_DM_ANALYTCS.NDAU_CROSS_BC'))

In [156]:
2.941468e+07

29414680.0

In [157]:
vol_days['total_volume'].mean()

12784362.098432938

In [158]:
vol_days['total_volume'].median()

3849935.709041893

In [100]:
portfolio_members['coins'].str.len()

2023-03-25    21
2023-03-26    23
2023-03-27    19
2023-03-28    23
2023-03-29    21
Name: coins, dtype: int64